In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

C:\Users\natal\AppData\Local\Temp\ipykernel_16716\3504193263.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
mobile_modelling = pd.read_csv('../Data/Data_modelling/mobile_modelling.csv')

X = mobile_modelling.iloc[:, 0:-1]
y = mobile_modelling.iloc[:, -1]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, stratify=y, test_size=0.3, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_val, y_val, stratify=y_val, test_size=0.3, random_state=42
)

mobile_df = pd.concat([X_train, y_train], axis=1)

# variables with importance of more than 0.02 
columns_to_drop_1 = ['touch_screen', 'blue', 'dual_sim', 'four_g', 'wifi', 'three_g']
X_train_r = X_train.drop(columns=columns_to_drop_1)
X_val_r = X_val.drop(columns=columns_to_drop_1)


# Bagging (Bootstrap Aggregating)
designed to improve the stability and accuracy of machine learning algorithms used in statistical classification and regression. It decreases the variance and helps to avoid overfitting. It is usually applied to decision tree methods

we're testing Bagging method with two tree models - DecisionTreeClassifier and RandomForestClassifier

in 2 variants: with all the columns and with columns having importance greater than 0.02

In [8]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [13]:
# Creating 2 models
model_dt = DecisionTreeClassifier(random_state=42)
model_rf = RandomForestClassifier(random_state=42)

# Bagging for first model
bagging_model_dt = BaggingClassifier(estimator=model_dt, n_estimators=10, random_state=42)
bagging_model_dt.fit(X_train, y_train)

bagging_model_dt_r = BaggingClassifier(estimator=model_dt, n_estimators=10, random_state=42)
bagging_model_dt_r.fit(X_train_r, y_train)

# Bagging for second model
bagging_model_rf = BaggingClassifier(estimator=model_rf, n_estimators=10, random_state=42)
bagging_model_rf.fit(X_train, y_train)

bagging_model_rf_r = BaggingClassifier(estimator=model_rf, n_estimators=10, random_state=42)
bagging_model_rf_r.fit(X_train_r, y_train)

# Checking predictions on validation datasets
predictions_model_dt = bagging_model_dt.predict(X_val)
predictions_model_dt_r = bagging_model_dt_r.predict(X_val_r)

predictions_model_rf = bagging_model_rf.predict(X_val)
predictions_model_rf_r = bagging_model_rf_r.predict(X_val_r)

# Printing classification report
print("Bagging with DecisionTreeClassifier - all columns")
print(classification_report(y_val, predictions_model_dt))

print("Bagging with DecisionTreeClassifier - important columns")
print(classification_report(y_val, predictions_model_dt_r))

print("\nBagging with RandomForestClassifier - all columns")
print(classification_report(y_val, predictions_model_rf))

print("\nBagging with RandomForestClassifier - important columns")
print(classification_report(y_val, predictions_model_rf_r))

Bagging with DecisionTreeClassifier - all columns
              precision    recall  f1-score   support

           0       0.96      0.92      0.94        73
           1       0.78      0.84      0.81        74
           2       0.70      0.79      0.74        73
           3       0.94      0.78      0.85        74

    accuracy                           0.83       294
   macro avg       0.84      0.83      0.84       294
weighted avg       0.84      0.83      0.84       294

Bagging with DecisionTreeClassifier - important columns
              precision    recall  f1-score   support

           0       0.93      0.90      0.92        73
           1       0.79      0.81      0.80        74
           2       0.72      0.82      0.77        73
           3       0.94      0.81      0.87        74

    accuracy                           0.84       294
   macro avg       0.84      0.84      0.84       294
weighted avg       0.84      0.84      0.84       294


Bagging with RandomFore

# Boosting
we're checking Boosting method with:
    AdaBoost
    XGBoost
    CatBoost

TODO ogarnąć xgboost i catboost

In [24]:
#!pip install xgboost
#!pip install catboost

In [25]:
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [26]:
# Function for training and evaluating the model
def train_and_evaluate_model(model, X_train, X_val, y_train, y_val):
    model.fit(X_train, y_train)
    predictions = model.predict(X_val)
    print(classification_report(y_val, predictions))

# Modeling with AdaBoost for all columns
print("AdaBoost for all columns:")
ada_model = AdaBoostClassifier(random_state=42, algorithm='SAMME')
train_and_evaluate_model(ada_model, X_train, X_test, y_train, y_test)

# Modeling with AdaBoost for selected columns
print("\nAdaBoost for selected columns:")
train_and_evaluate_model(ada_model, X_train_r, X_val_r, y_train, y_val)

# Modeling with XGBoost for all columns
print("\nXGBoost for all columns:")
xgb_model = XGBClassifier(random_state=42)
train_and_evaluate_model(xgb_model, X_train, X_test, y_train, y_test)

# Modeling with XGBoost for selected columns
print("\nXGBoost for selected columns:")
train_and_evaluate_model(xgb_model, X_train_r, X_val_r, y_train, y_val)

# Modeling with CatBoost for all columns
print("\nCatBoost for all columns:")
cat_model = CatBoostClassifier(random_state=42, verbose=0)
train_and_evaluate_model(cat_model, X_train, X_test, y_train, y_test)

# Modeling with CatBoost for selected columns
print("\nCatBoost for selected columns:")
train_and_evaluate_model(cat_model, X_train_r, X_val_r, y_train, y_val)


AdaBoost for all columns:
              precision    recall  f1-score   support

           0       0.92      0.72      0.81        32
           1       0.66      0.68      0.67        31
           2       0.40      0.72      0.52        32
           3       0.33      0.13      0.19        31

    accuracy                           0.56       126
   macro avg       0.58      0.56      0.54       126
weighted avg       0.58      0.56      0.55       126


AdaBoost for selected columns:
              precision    recall  f1-score   support

           0       0.88      0.58      0.69        73
           1       0.60      0.74      0.67        74
           2       0.41      0.68      0.52        73
           3       0.41      0.19      0.26        74

    accuracy                           0.55       294
   macro avg       0.58      0.55      0.53       294
weighted avg       0.58      0.55      0.53       294


XGBoost for all columns:
              precision    recall  f1-score   

# Bayes optimization

In [1]:
! pip install scikit-optimize

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/107.7 kB ? eta -:--:--
   ---------------------- ----------------- 61.4/107.7 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 107.7/107.7 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/138.7 kB ? eta -:--:--
   ----------------------------------- ---- 122.9/138.7 kB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 138.7/138.7 kB 1.6 MB/s eta 0:00:00


### SVM

In [6]:
from skopt import BayesSearchCV
from sklearn.svm import SVC

opt = BayesSearchCV(
    SVC(),
    {
        'degree': (1, 8),  # integer valued parameter
        'kernel': ['linear', 'poly', 'rbf'],  # categorical parameter
    },
    n_iter=10,
    cv=3
)

opt.fit(X_train, y_train)

print("val. score: %s" % opt.best_score_)
print("test score: %s" % opt.score(X_val, y_val))

val. score: 0.9653070924247825
test score: 0.9693877551020408


### Random Forest Classifier

In [11]:
# Define the optimizer with BayesSearchCV
opt_rf = BayesSearchCV(
    RandomForestClassifier(),
    {
        'n_estimators': (10, 1000),  # Number of trees in the forest
        'max_depth': (1, 20),  # Maximum depth of the tree
        'min_samples_split': (2, 10),  # Minimum number of samples required to split a node
        'min_samples_leaf': (1, 10),  # Minimum number of samples required at each leaf node
        'max_features': (1, len(X_train.columns)),  # Number of features to consider when looking for the best split
    },
    n_iter=10,  # Number of optimization iterations
    cv=3
)

# Fit the optimizer to the training data
opt_rf.fit(X_train, y_train)

# Print the best validation score for Random Forest
print("Random Forest - val. score: %s" % opt_rf.best_score_)

# Print the test score for Random Forest
print("Random Forest - test score: %s" % opt_rf.score(X_val, y_val))

Random Forest - val. score: 0.86024027066409
Random Forest - test score: 0.8707482993197279


### Decision Tree Classifier

In [13]:
# Define the optimizer with BayesSearchCV
opt_dt = BayesSearchCV(
    DecisionTreeClassifier(),
    {
        'max_depth': (1, 20),  # Maximum depth of the tree
        'min_samples_split': (2, 10),  # Minimum number of samples required to split a node
        'min_samples_leaf': (1, 10),  # Minimum number of samples required at each leaf node
        'max_features': (1, len(X_train.columns)),  # Number of features to consider when looking for the best split
    },
    n_iter=10,  # Number of optimization iterations
    cv=3
)

# Fit the optimizer to the training data
opt_dt.fit(X_train, y_train)

# Print the best validation score for Decision Tree
print("Decision Tree - val. score: %s" % opt_dt.best_score_)

# Print the test score for Decision Tree
print("Decision Tree - test score: %s" % opt_dt.score(X_val, y_val))

Decision Tree - val. score: 0.824499853035903
Decision Tree - test score: 0.8299319727891157
